In [ ]:
import pandas as pd
import numpy as np
import pickle
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
df = pd.read_csv('../Data/Interim/hydrated_Tweet200316.csv', index_col=0)

In [ ]:
# Clean tweets
df['CleanTweet'] = [re.sub("[^a-zA-Z0-9\s,.-_´&%'\":€$£!?']", '',  re.sub(' http\S+', '', re.sub('\s',' ', tw))).replace(u'\xa0', u' ').lower() if isinstance(tw, str) else '' for tw in df.tweet]

# Get bigrams
bigrams_tweets = [list(nltk.bigrams(tw)) for tw in df['CleanTweet']]

In [ ]:
with open('../Data/Processed/Bigram_tweets.pickle', 'wb') as handle:
    pickle.dump(bigrams_tweets, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
unique_chars = sorted(''.join(set(''.join(df['CleanTweet'])))) # Assume all characters are in tweets from df1

unique_bigrams = [x+y for x in unique_chars for y in unique_chars]

bigram_mapper = dict(zip(unique_bigrams, range(len(unique_bigrams))))

In [ ]:
Vector = CountVectorizer()

In [ ]:
X = Vector.fit_transform(df['CleanTweet'].values)

In [ ]:
X.shape

In [ ]:
lda = LatentDirichletAllocation(n_components=5, verbose = 2, n_jobs = 3)
lda.fit(X[:10000])

In [ ]:
topics = lda.transform(X[:10000]).argmax(axis = 1)

In [ ]:
y = np.bincount(topics)
ii = np.nonzero(y)[0]
dict(zip(ii,y[ii]))

In [ ]:
lda.score(X[:10000])

In [ ]:
lda.perplexity(X[124])

In [ ]:
with open('../Data/Processed/Embedding.pickle', 'rb') as f:
    embed = pickle.load(f)

In [ ]:
embed['trump']

In [ ]:
from gensim.test.utils import common_corpus

In [ ]:
from gensim.models import LdaModel

In [ ]:
lda = LdaModel(X[:2000].todense(), num_topics=10)